### Dependencies

In [9]:
import cv2
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

from tensorflow.python import pywrap_tensorflow

sys.path.append('..')

import video_input as vi
import model as model
import utils as utils

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
params = utils.yaml_to_dict('../config.yml')
params['data_dir'] = os.path.join('..', params['data_dir'])
params['videos_folder'] = os.path.join('..', params['videos_folder'])

In [11]:
params

{'batch_size': 15,
 'buffer_size': 50,
 'classes_amount': 101,
 'data_dir': '../data',
 'eval_steps': 10,
 'feature_map_folder': 'data/feature_maps',
 'feature_maps_size': [5, 5, 1536],
 'json_data_path': 'data/activity_net.v1-2.min.json',
 'json_metadata_path': 'data/training_meta_data_reduced.json',
 'keep_checkpoint_max': 3,
 'label_feature_maps_size': [101],
 'learning_rate': 0.0001,
 'log_step_count_steps': 20,
 'max_frames': [15],
 'max_steps': 21000,
 'model': 'gap',
 'model_dir': '.temp/checkpoints',
 'num_epochs': 20,
 'resize': [224, 224],
 'save_checkpoints_steps': 20,
 'save_summary_steps': 20,
 'shuffle': True,
 'skip_frames': 10,
 'start_delay_secs': 10,
 'taxonomy_level': 2,
 'temp_dir': '.temp',
 'throttle_secs': 10,
 'videos_folder': '../data/videos',
 'weight_factor': 8}

### Set test image

In [ ]:
img_path = os.path.join('..',params['temp_dir'], 'test_img.jpg')
img_width = 224
img_height = 224

test_img = cv2.imread(img_path)
print('Original Size',test_img.shape)

img_resize = cv2.resize(test_img,(img_width,img_height))
img_resize = img_resize[None, ...]

img_normalized = (img_resize / 255.0)
print('Resized and normalized image', img_normalized.shape)

### Load pretrained model

In [12]:
pretrain_model = model._initialize_pretrained_model()

### Make predictions with different models

In [13]:
params['json_data_path'] = os.path.join('..',params['json_data_path'])
params['json_metadata_path'] = os.path.join('..',params['json_metadata_path'])

In [6]:
video_gen = vi.all_data_videos(params)

In [ ]:
particular_frames = next(video_gen)[0]
particular_frames.shape

In [ ]:
#particular_frames = particular_frames[None, ...]

In [ ]:
block4_pool_features = pretrain_model.predict(particular_frames)

In [ ]:
block4_pool_features.shape

In [ ]:
x = model.gap_module(block4_pool_features)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    elements = sess.run(x)

In [ ]:
elements[0,...]

In [ ]:
x = model.dense_module(block4_pool_features)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    elements = sess.run(x)

In [ ]:
elements[0,...]

## Preprocessing data and make logits

In [14]:
all_data_videos = vi.all_data_videos(params)

In [15]:
videos_dict = dict()
current_video = 'not a video'
steps_videos = 0

for data in all_data_videos:
    steps_videos += data[2] != current_video
    
    if data[2] != current_video:
        print('current video: ', data[2])
        
    current_video = data[2]
    
    if steps_videos == 2:
        break

    image_frames = data[0]
    image_label = data[1]
    image_video_id = data[2]
    
    print(image_label.shape)
    
    try:
        image_feature_map = pretrain_model.predict(image_frames)
    except:
        continue

    image_video_name = image_video_id + '_batch_1.npz'
    image_feature_path = os.path.join(params['data_dir'] + '/feature_maps/training/', image_video_name)
    batch_count = 1
    while os.path.isfile(image_feature_path):
        batch_count += 1
        image_video_name = image_video_id + '_batch_' + str(batch_count) + '.npz'
        image_feature_path = os.path.join(params['data_dir'] + '/feature_maps/training/', image_video_name)    
        
    np.savez(image_feature_path, feature_map=image_feature_map, label = image_label)

current video:  1Y1pKGFm-pQ
(15, 34)
(15, 34)
(15, 34)
(15, 34)
(15, 34)
(15, 34)
(9, 34)
current video:  0p3JN4KJsUs


In [ ]:
image_feature_file = np.load(os.path.join(params['data_dir'],'test.npz'))

In [ ]:
image_feature_file['feature_map'].shape

In [ ]:
image_feature_file['label']

In [ ]:
1536*15